In [ ]:
# CRAWLER

import requests
from bs4 import BeautifulSoup
import time
import os
import re

# cria uma pasta chamada "wikipedia_pessoas" se não existir ainda
os.makedirs("wikipedia_pessoas", exist_ok=True)

# site da wikipedia em pt
URL_BASE = "https://pt.wikipedia.org"
PAGINA_INICIAL = "/"  # começa na página inicial (raiz)

# listas e contadores pra controlar
visitados = set()  # guarda páginas que já visitou
a_visitar = [PAGINA_INICIAL]  # começa com a página inicial
pessoas_coletadas = 0  # quantas pessoas já salvou
MAX_PESSOAS = 1000  # limite máximo
total_visitas = 0  # quantas páginas já acessou

# cabeçalho só pra fingir que somos um navegador normal
cabecalhos = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# função pra tentar descobrir se a página é de uma pessoa
def eh_pagina_de_pessoa(sopa):
    # procura tabelinha de "infobox" que normalmente tem em páginas de pessoas
    infobox = sopa.find("table", class_=lambda c: (
        (isinstance(c, str) and "infobox" in c) or
        (isinstance(c, list) and any("infobox" in cls for cls in c))
    ))
    if not infobox:
        return False

    # pega os nomes das colunas da infobox
    rotulos = [th.get_text(strip=True).lower() for th in infobox.find_all("th")]

    # coisas que normalmente só aparecem em páginas de pessoas
    rotulos_pessoa = [
        'nascimento', 'data de nascimento', 'ocupação',
        'nome completo', 'nacionalidade', 'morte', 'cidadania'
    ]
    # coisas que normalmente NÃO são pessoas
    rotulos_nao_pessoa = [
        'localização', 'população', 'idioma',
        'extinção', 'capital', 'território'
    ]

    # se tem coisas de pessoa e não tem coisas que não são de pessoa
    if any(rotulo in rotulos for rotulo in rotulos_pessoa):
        if any(rotulo in rotulos for rotulo in rotulos_nao_pessoa):
            return False
        return True

    # caso não ache pela infobox, tenta pelas categorias da página
    div_categorias = sopa.find("div", {"id": "mw-normal-catlinks"})
    if div_categorias:
        categorias = [li.get_text(strip=True).lower() for li in div_categorias.find_all("li")]
        palavras_pessoa = [
            'nascimentos', 'homens', 'mulheres', 'cantores', 'escritores', 'atores', 'políticos', 'biografia'
        ]
        # se achar alguma dessas palavras nas categorias, considera pessoa
        if any(any(palavra in cat for palavra in palavras_pessoa) for cat in categorias):
            return True

    return False


# limpa caracteres estranhos do nome de arquivo
def limpar_nome_arquivo(nome):
    return re.sub(r'[\\/*?:"<>|]', "", nome)

# começa o loop principal pra ir visitando as páginas
while a_visitar and pessoas_coletadas < MAX_PESSOAS:
    caminho_atual = a_visitar.pop(0)  # pega a próxima página da lista
    if caminho_atual in visitados:  # se já foi visitada, pula
        continue

    # monta a URL completa (se for raiz, coloca só "/")
    if caminho_atual == "/":
        url = URL_BASE + "/"
    else:
        url = URL_BASE + caminho_atual

    total_visitas += 1
    print(f"{total_visitas} Visitando ({pessoas_coletadas}/{MAX_PESSOAS}): {url}")

    # tenta acessar a página
    try:
        resp = requests.get(url, headers=cabecalhos)
        if resp.status_code != 200:  # se não deu certo, pula
            print(f"Erro ao acessar a página: {resp.status_code}, pulando.")
            continue
    except Exception as e:
        print(f"Erro na requisição: {e}")
        continue

    # transforma o HTML em objeto que dá pra analisar
    sopa = BeautifulSoup(resp.text, "html.parser")
    visitados.add(caminho_atual)

    # se a página for de pessoa
    if eh_pagina_de_pessoa(sopa):
        titulo = sopa.find("h1", id="firstHeading")
        if titulo:
            nome_pessoa = limpar_nome_arquivo(titulo.text.strip())
        else:
            nome_pessoa = f"pessoa_desconhecida_{pessoas_coletadas}"

        # salva o HTML da página na pasta
        nome_arquivo = f"wikipedia_pessoas/{nome_pessoa}.html"
        with open(nome_arquivo, "w", encoding="utf-8") as f:
            f.write(resp.text)

        pessoas_coletadas += 1

    # pega todos os links da página e adiciona na lista de páginas a visitar
    for tag_link in sopa.find_all("a", href=True):
        href = tag_link['href']
        if href.startswith("/wiki/") and ":" not in href and href not in visitados and href not in a_visitar:
            a_visitar.append(href)

    # espera um pouco pra não dar sobrecarga no servidor da wiki
    time.sleep(0.6)

print(f"Coletadas {pessoas_coletadas} páginas de pessoas em {total_visitas} visitas.")

In [ ]:
# GRAFO

import os
from bs4 import BeautifulSoup
from collections import defaultdict, deque

PASTA = "wikipedia_pessoas"

nomes_para_links = {}
links_para_nomes = {}

def normalizar_href(href):
    href = href.split("#")[0]
    href = href.split("?")[0]
    return href.strip().lower()

# pega os nomes das pessoas e seus links
for arq in os.listdir(PASTA):
    if not arq.endswith(".html"):
        continue
    caminho = os.path.join(PASTA, arq)
    html = open(caminho, "r", encoding="utf-8").read()
    sopa = BeautifulSoup(html, "html.parser")

    titulo = sopa.find("h1", id="firstHeading")
    if not titulo:
        continue
    nome = titulo.text.strip()

    link = None
    for a in sopa.select("link[rel=canonical]"):
        href = a.get("href")
        if href and "/wiki/" in href:
            link = href[href.find("/wiki/"):]
            break

    if link:
        link = normalizar_href(link)
        nomes_para_links[nome] = link
        links_para_nomes[link] = nome

# monta o grafo
arestas = defaultdict(set)

for arq in os.listdir(PASTA):
    if not arq.endswith(".html"):
        continue
    caminho = os.path.join(PASTA, arq)
    html = open(caminho, "r", encoding="utf-8").read()
    sopa = BeautifulSoup(html, "html.parser")

    titulo = sopa.find("h1", id="firstHeading")
    if not titulo:
        continue
    nome_a = titulo.text.strip()
    link_a = nomes_para_links.get(nome_a)
    if not link_a:
        continue

    for a in sopa.find_all("a", href=True):
        href = a["href"]
        if not href.startswith("/wiki/"):
            continue
        if ":" in href:
            continue
        href = normalizar_href(href)
        nome_b = links_para_nomes.get(href)
        if nome_b and nome_b != nome_a:
            arestas[nome_a].add(nome_b)

# mostra resumo
print("Grafo construído:", len(nomes_para_links), "pessoas,", sum(len(v) for v in arestas.values()), "conexões.")

# busca caminho com bfs
def menor_caminho_bfs(arestas, origem, destino):
    if origem not in arestas and origem != destino:
        return None
    fila = deque([origem])
    anteriores = {origem: None}

    while fila:
        atual = fila.popleft()
        if atual == destino:
            break
        for viz in arestas.get(atual, []):
            if viz not in anteriores:
                anteriores[viz] = atual
                fila.append(viz)

    if destino not in anteriores:
        return None

    caminho = []
    cur = destino
    while cur is not None:
        caminho.append(cur)
        cur = anteriores[cur]
    caminho.reverse()
    return caminho



In [ ]:
# INTERFACE
pessoa_origem = input("Nome da pessoa de origem: ").strip()
pessoa_destino = input("Nome da pessoa de destino: ").strip()

caminho = menor_caminho_bfs(arestas, pessoa_origem, pessoa_destino)
if caminho is None:
    print("Não há conexão encontrada entre as pessoas informadas.")
else:
    graus = len(caminho) - 1
    print(f"Grau de separação: {graus}")
    print(" -> ".join(caminho))